In [97]:
import pandas as pd

In [98]:
df = pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [99]:
df=df.drop(labels=['id'],axis=1)

In [100]:
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [101]:
# Segregating numerical and categorical variables
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [102]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [103]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [104]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [105]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [106]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [107]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [108]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [109]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [110]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [111]:
X_train.describe()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
count,1.355010e+05,1.355010e+05,1.355010e+05,1.355010e+05,1.355010e+05,1.355010e+05,1.355010e+05,1.355010e+05,1.355010e+05
mean,-3.429458e-17,-3.223848e-15,-1.275061e-15,2.175136e-16,-2.340841e-16,2.633446e-16,-5.002603e-17,4.981628e-17,4.079691e-17
std,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00
min,-1.278098e+00,-8.991863e+00,-4.292180e+00,-5.153411e+00,-5.190675e+00,-5.122673e+00,-3.150769e+00,-1.551019e+00,-1.979798e+00
25%,-8.457284e-01,-4.795040e-01,-6.428619e-01,-9.165526e-01,-9.176705e-01,-9.202756e-01,-1.138347e+00,-9.350706e-01,-6.467863e-01
50%,-1.971742e-01,7.564991e-02,-1.215308e-01,-6.078784e-03,-1.378763e-03,-7.341025e-03,-1.321357e-01,2.968257e-01,1.971957e-02
75%,5.162355e-01,5.382781e-01,3.998003e-01,7.150886e-01,7.153247e-01,7.172103e-01,8.740755e-01,9.127739e-01,6.862254e-01
max,5.855999e+00,9.050637e+00,6.655773e+00,3.545671e+00,3.890593e+00,4.023424e+01,8.740755e-01,2.144670e+00,2.685743e+00


In [112]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [115]:
regressor=LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression()

In [116]:
def model_evaluate(real,pred):
    mae=mean_absolute_error(real,pred)
    mse=mean_squared_error(real,pred)
    r2=r2_score(real,pred)
    return mae,mse,r2

In [117]:
models={
    "LinearRegression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "ElasticNet":ElasticNet()
}

In [118]:
trainer_model_list=[]
model_list=[]
r2_list=[]

In [122]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    Y_pred=model.predict(X_test)
    mae,mse,r2=model_evaluate(y_test,Y_pred)
    print(list(models.keys())[i])

    print("Model Performance")
    print("mean_absolute_error ",mae)
    print("mean_squared_error ",mse)
    print("r2_score ",r2)
    print("\n")

LinearRegression
Model Performance
mean_absolute_error  674.0255115796832
mean_squared_error  1028002.7598132559
r2_score  0.9368908248567511


Lasso
Model Performance
mean_absolute_error  675.0716923362162
mean_squared_error  1027949.4559693959
r2_score  0.9368940971841704


Ridge
Model Performance
mean_absolute_error  674.0555800798253
mean_squared_error  1028005.2293677655
r2_score  0.9368906732505938


ElasticNet
Model Performance
mean_absolute_error  1060.7368759154729
mean_squared_error  2351365.382289642
r2_score  0.8556494831165181


